# MNIST

約6000枚/数字 の画像のデータセットから学習し、識別する

## 注

loadしたmnist.pklのdatasetは'image'が(データ数,784)の行列'label'が(データ数,10)の0or1行列

w1:一つ目の重み関数、b1:一つ目の閾値、w2:二つ目の重み関数、b2:二つ目の閾値

learning_rateを上げすぎると学習できない（更新にSGDを使用しているため）

In [2]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import time

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def softmax(a):
    a=a.T
    y = np.exp(a-np.max(a,axis=0))/np.sum(np.exp(a-np.max(a,axis=0)),axis=0)
    return y.T

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


In [4]:
class Classification:
    def __init__(self, input_size = 28*28, hidden1_size = 100, output_size = 10, 
                 weight_init_std = 0.01):
        self.load()
        
        # 重み関数などを格納するdict
        self.params = {}
        self.params['w1'] = weight_init_std*np.random.randn(input_size, hidden1_size)
        self.params['b1'] = np.zeros(hidden1_size)
        self.params['w2'] = weight_init_std*np.random.randn(hidden1_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        self.dw1 = None
    
    def train(self, batch_size=200, n_epoch=10,leaning_rate = 0.0001):
        datlen = self.dataset['label'].shape[0]
        for n in range(n_epoch):# データセットを何周するか
            batch_mask = np.random.permutation(datlen)# 一回のバッチでデータのどこをとってくるのかランダムに並べ替え
            for b in range(datlen//batch_size):# バッチ処理
                batch_img = self.dataset['image'][batch_mask[b*batch_size:(b+1)*batch_size],:]
                batch_label = self.dataset['label'][batch_mask[b*batch_size:(b+1)*batch_size],:]
                grads = self.gradient(batch_img, batch_label)
                for key in ('w1','b1','w2','b2'):
                    self.params[key] -= leaning_rate * grads[key]
            if 0<=n<10 or n_epoch-5<=n:
                print('trial:{}, Correct Rate:{}%'.format(n+1,self.correct_rate))
        #print('w1:{}, b1:{}, w2:{}, b2:{}'.format(self.params['w1'],self.params['b1'],self.params['w2'],self.params['b2']))
        return
        
    def predict(self,batchdat):# imgdataからlabelを予測
        w1,w2 = self.params['w1'], self.params['w2']
        b1,b2 = self.params['b1'], self.params['b2']
        
        self.batchdat = batchdat
        self.a1 = np.dot(batchdat,w1) + b1
        self.z1 = sigmoid(self.a1)
        self.a2 = np.dot(self.z1,w2) + b2
        self.y = softmax(self.a2)
        return 
    
    def gradient(self, x, t):# 勾配
        L = self.loss(x, t)
        grads = {}# batch未対応
        db2 = (self.y-t)*self.y*(1-self.y)
        grads['b2'] = np.sum(db2,axis=0)
        grads['w2'] = np.dot(self.z1.T, db2)
        db1 = np.dot(db2, self.params['w2'].T)*self.z1*(1-self.z1)
        grads['b1'] = np.sum(db1,axis=0)
        grads['w1'] = np.dot(self.batchdat.T, db1)
        return grads
    
    def loss(self, x, t):# 損失関数
        self.predict(x)
        self.correct(self.y,t)
        return mean_squared_error(self.y,t)
        
    def load(self):
        print('loading...')
        with open(r'./train_img/mnist.pkl', 'rb') as f:
            self.dataset= pickle.load(f)
        return
    
    def correct(self, y, t):# 正答率(データセットを一周するときの最後のバッチで計算)
        ypre = np.argmax(y,axis=1)
        ans = np.argmax(t,axis=1)
        self.correct_rate = 100*sum(ypre==ans)/t.shape[0]
        return
        
        
        

In [10]:
cls = Classification()
start = time.time()
cls.train()
print('finish!\n 学習時間は{}sだよ(^^)'.format(time.time()-start))

loading...
trial:1, Correct Rate:59.5%
trial:2, Correct Rate:66.0%
trial:3, Correct Rate:74.0%
trial:4, Correct Rate:76.5%
trial:5, Correct Rate:83.0%
trial:6, Correct Rate:85.5%
trial:7, Correct Rate:85.0%
trial:8, Correct Rate:83.5%
trial:9, Correct Rate:91.5%
trial:10, Correct Rate:94.5%
finish!
 学習時間は13.038674116134644sだよ(^^)


In [5]:
i = np.zeros(3)

w = np.array([[1],[3]])
print(i[0:1])


[0.]
